In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import gsw

from scipy.integrate import solve_ivp

In [2]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [3]:
lat = 35+8.4585/60
bini = 15

Smooth ADCP time series
* 30-min running median to remove spikes
* 60-min running mean to smooth

In [4]:
ds['Eas_sm'] = ds['Eas'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()
ds['Nor_sm'] = ds['Nor'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()

In [5]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Eas_sm'][:,bini]);
plt.plot(ds['time'],ds['Eas'][:,bini],'--');
plt.plot(ds['time'],ds['Eas_sm'][:,1],'-');
#plt.plot(ds['time'],ds['Eas'][:,1]);
plt.ylabel('m/s')
plt.title('eastward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'eastward velocity')

In [6]:
plt.figure(figsize=(8,4))
plt.plot(ds['time'],ds['Nor_sm'][:,bini]);
plt.legend(['Bin 15','Bin 1'])
plt.ylabel('m/s')
plt.title('northward velocity')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'northward velocity')

### Test with limited obs

In [7]:
zf = ds['binheight'][bini] # upper boundary height
zobs = np.array(ds['binheight'][0:(bini+1)])

In [8]:
zo = 0.03 # roughness length
i = 1j
kappa = 0.41 # Von Karman constant
f = gsw.f(lat)

z = np.linspace(zo,zf,501)
dz = np.diff(z)
deltaz = dz[0]

In [9]:
gi = np.isfinite(ds['Eas_sm'][:,0])
tobs = np.array((ds['time'][gi] - ds['time'][gi][0])/np.timedelta64(1,'s')) # array of times in seconds
deltat = tobs[1]-tobs[0]

In [10]:
wobs = np.array(ds['Eas_sm'][gi,0:(bini+1)]+i*ds['Nor_sm'][gi,0:(bini+1)])

w0_obs = np.interp(z,zobs[1:bini],wobs[0,1:bini])[1:-1]
w0 = np.concatenate([np.real(w0_obs),
                           np.imag(w0_obs)])

dwobsdt = np.gradient(wobs,deltat,axis=0)

In [11]:
def dwdt_bbl(t, w_in):
    # Note that w_in is real and complex parts concatenates
    # need to combine into complex numbers
    # Scipy docs say that ode solvers RK45 and BDF can be used in complex plane
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.BDF.html#scipy.integrate.BDF
    # but perhaps this does not work for *arrays* of complex numbers?
    # Test this and check Numpy Github issues for similar problems
    
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = np.interp(t,tobs,wobs[:,bini])
    dwinfdt = np.interp(t,tobs,dwobsdt[:,bini])

    F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[1])/deltaz

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     (kappa*ustar/deltaz**2)*z[1:-1]*wall[2:] +
                     (-i*f - (kappa*ustar/deltaz**2)*(z[1:-1]+z[:-2]))*wall[1:-1] +
                     (kappa*ustar/deltaz**2)*z[:-2]*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [12]:
a = solve_ivp(dwdt_bbl, t_span=[tobs[0],86400], y0 = w0, method='BDF')

In [13]:
N = int(np.shape(a.y)[0]/2)

t_sol = a.t
w_sol = a.y[:N,:]+i*a.y[N:,:]

In [14]:
plt.figure()
plt.subplot(211)
plt.plot(t_sol,np.real(w_sol).T,color='gray')
plt.plot(tobs[0:12*24],np.real(wobs[0:12*24,bini]))

plt.subplot(212)
plt.plot(t_sol,np.imag(w_sol).T,color='gray')
plt.plot(tobs[0:12*24],np.imag(wobs[0:12*24,bini]))

FigureCanvasNbAgg()

### Test with steady forcing

In [15]:
winf_steady = 0.04+i*0
w0_steady = np.concatenate([np.real(winf_steady)*np.ones(N),
                           np.imag(winf_steady)*np.ones(N)])

def dwdt_bbl_steady(t, w_in):
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = winf_steady
    dwinfdt = 0+i*0

    F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[1])/deltaz

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     (kappa*ustar/deltaz**2)*z[1:-1]*wall[2:] +
                     (-i*f - (kappa*ustar/deltaz**2)*(z[1:-1]+z[:-2]))*wall[1:-1] +
                     (kappa*ustar/deltaz**2)*z[:-2]*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    #print(dwdt)
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

In [16]:
modsteady = solve_ivp(dwdt_bbl_steady, t_span=[0,2*86400], 
                      y0 = w0_steady, method='BDF')

In [17]:
t_sol_steady = modsteady.t
w_sol_steady = modsteady.y[:N,:]+i*modsteady.y[N:,:]

In [18]:
plt.figure()
plt.subplot(121)
plt.plot(np.real(w_sol_steady[:,-1]),z[1:-1]);
plt.plot(np.imag(w_sol_steady[:,-1]),z[1:-1]);
#plt.plot(tobs[0:12*24],wobs[0:12*24,bini])

FigureCanvasNbAgg()

In [19]:
ustar_steady = kappa*zo*np.abs(w_sol_steady[1,-1])/deltaz

In [20]:
ustar_steady

0.003753643680807231